In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
import aux_functions


#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [2]:
df_preds= pd.read_csv('results/forecast_quantiles_death_v3.csv')

#df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.forecast_date = pd.to_datetime(df_preds.forecast_date)
df_preds.target_end_date = pd.to_datetime(df_preds.target_end_date)


#df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

,model,forecast_date,target_end_date,horizon,lower_95,upper_95,lower_50,upper_50,lower_98,lower_90,...,upper_10,upper_20,upper_30,upper_40,upper_60,upper_70,upper_80,upper_90,upper_98,value
0,AIpert-pwllnod,2021-01-11,2021-01-23,1,15585.124720,30107.231540,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21731.0
1,AIpert-pwllnod,2021-01-11,2021-01-30,2,15873.523490,36138.769513,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21862.0
2,AIpert-pwllnod,2021-01-11,2021-02-06,3,16301.068910,44059.836613,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21394.0
3,AIpert-pwllnod,2021-01-11,2021-02-13,4,16888.656010,54505.710936,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17536.0
4,AIpert-pwllnod,2021-01-18,2021-01-30,1,17927.563802,33673.501984,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21862.0


In [3]:
len(df_preds)

11165

In [4]:
df_preds = df_preds.dropna()
df_preds = df_preds.reset_index(drop=True)

In [5]:
len(df_preds)

10933

In [6]:
11165-10933

232

### Estimate the log normal parameters from the CDF: 

Fixed conditions for the entire method:

In [7]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98)

p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [8]:
ps

[0.010000000000000009,
 0.025000000000000022,
 0.04999999999999999,
 0.09999999999999998,
 0.15000000000000002,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9,
 0.95,
 0.975,
 0.99]

In [9]:
the_pos

['lower_95',
 'lower_50',
 'lower_98',
 'lower_90',
 'lower_80',
 'lower_70',
 'lower_60',
 'lower_40',
 'lower_30',
 'lower_20',
 'lower_10',
 'upper_95',
 'upper_50',
 'upper_10',
 'upper_20',
 'upper_30',
 'upper_40',
 'upper_60',
 'upper_70',
 'upper_80',
 'upper_90',
 'upper_98']

In [10]:
def process_row(row):
    '''
    Estimate the lognormal parameters for each row in the dataframe
    '''
    #the_pos =  ['lower_98', 'lower_95', 'lower_90', 'lower_80', 'lower_70', 'lower_60', 'lower_50', 'lower_40',
    #            'lower_30', 'lower_20', 'lower_10']

    #ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    out = out.loc[out.xi > 0]
    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = aux_functions.fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=0
    )
    
    return cdf_fit

def process_row2(row):
    #the_pos =  ['upper_50', 'upper_10', 'upper_20', 'upper_30', 'upper_40', 'upper_60', 'upper_70', 'upper_80', 'upper_90', 'upper_95', 'upper_98']

    #ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=1
    )
    
    return cdf_fit

In [11]:
%%time

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1, result_type = 'expand')

df_preds.head()

CPU times: total: 1h 22min 8s
Wall time: 1h 28min 45s


,model,forecast_date,target_end_date,horizon,lower_95,upper_95,lower_50,upper_50,lower_98,lower_90,...,upper_30,upper_40,upper_60,upper_70,upper_80,upper_90,upper_98,value,mu,sigma
0,AIpert-pwllnod,2021-07-19,2021-07-31,1,595.955733,4341.901891,1521.826927,2810.933415,428.091306,822.916022,...,2466.539847,2632.180792,3009.982937,3241.999466,3533.929238,3966.613137,4778.256172,2571.0,7.627476,0.454140
1,AIpert-pwllnod,2021-07-19,2021-08-07,2,482.710172,5682.092035,1566.049852,3355.332615,342.360199,748.270963,...,2831.924726,3083.665134,3657.847238,4010.464978,4454.138612,5111.729773,6345.261427,3668.0,7.718169,0.558158
2,AIpert-pwllnod,2021-07-19,2021-08-14,3,404.932573,7807.806832,1660.787331,4208.366189,281.569183,712.782261,...,3398.666823,3788.103155,4676.349006,5221.840136,5908.192535,6925.470183,8833.713869,4661.0,7.843916,0.679365
3,AIpert-pwllnod,2021-07-19,2021-08-21,4,352.113552,11195.813238,1810.056893,5541.739899,230.007093,709.501538,...,4269.847943,4881.582321,6276.856667,7133.725041,8211.861115,9809.821186,12807.328527,7304.0,8.010528,0.800023
4,AIpert-pwllnod,2021-07-26,2021-08-07,1,649.222301,3820.828929,1459.009347,2550.466220,466.013035,847.726697,...,2264.696295,2402.141372,2715.632972,2908.154993,3150.391657,3509.422884,4182.905755,3668.0,7.558697,0.413585


In [12]:
df_preds.isnull().sum()

model              0
forecast_date      0
target_end_date    0
horizon            0
lower_95           0
upper_95           0
lower_50           0
upper_50           0
lower_98           0
lower_90           0
lower_80           0
lower_70           0
lower_60           0
lower_40           0
lower_30           0
lower_20           0
lower_10           0
pred               0
upper_10           0
upper_20           0
upper_30           0
upper_40           0
upper_60           0
upper_70           0
upper_80           0
upper_90           0
upper_98           0
value              0
mu                 4
sigma              0
dtype: int64

In [13]:
col = 'sigma'
pos_inf_count = np.isposinf(df_preds[col]).sum()

# count only negative inf
neg_inf_count = np.isneginf(df_preds[col]).sum()

# count both +inf and -inf
total_inf_count = np.isinf(df_preds[col]).sum()

print("positive inf:", pos_inf_count)
print("negative inf:", neg_inf_count)
print("total inf:", total_inf_count)

positive inf: 0
negative inf: 0
total inf: 0


In [15]:
df_preds.to_csv('results/preds_CDF_death_weights_equal_v3.csv', index = False)